In [110]:
import numpy as np
from sklearn.preprocessing import normalize
from scipy.constants import epsilon_0 as ε_0, c, pi as π, e, hbar as ℏ
eV = e
from scipy.special import lpmn, sph_jn
from numpy import newaxis as nx

I = np.identity(3)

## Green's functions
All vectors are expected to be in cartesian coordinates. Arguments and return values are in form of arrays ("lists") to enable fast batch processing.
### Free space

In [23]:
# Transverse delta; invalit at r1 = r2
def δ_T(r1, r2):
    print("Not implemented")
    return

# Longitudinal delta; invalid at r1 = r2
def δ_L(r1, r2):
    print("Not implemented")
    return

def G_fs(r1, r2, ω):
    r12 = np.linalg.norm((r1-r2), axis=1)
    g = np.exp(1j * r12 * ω/c) / (4*π*r12)
    return np.outer(g, I)    

def K_fs(r1, r2, ω):
    print("Not implemented")
    return

### Spherical particle (Mie solution)
TODO

In [149]:
# r[:,0] = x = r sin θ cos φ
# r[:,1] = y = r sin θ sin φ
# r[:,2] = z = r cos θ
def spherical_vector_waves(nmax, k, r):
    # FIXME this is _wrong_ very near z axis – but it avoids nan 
    # and should give correct result exactly on z axis
    # k is single scalar, r is single 3D vector
    r.shape = (3) # FIXME do proper check
    ρ = np.linalg.norm(r)
    r̂ = r/ρ
    cosθ = r̂[2]
    sinθ = np.sqrt(1-cosθ*cosθ)
    cosφ = r̂[0]/sinθ if sinθ else 0 # avoid zero division here
    sinφ = r̂[1]/sinθ if sinθ else 1
    φ = np.arctan2(sinφ,cosφ)
    # TODO can those be views so I don't have to write the indices
    # explicitly?
    m = np.arange(nmax+1)
    n = m
    sinmφ = np.sin(m*φ)
    cosmφ = np.cos(m*φ)
    θ̂ = np.array([cosθ*cosφ,cosθ*sinφ,-sinθ])
    φ̂ = np.array([-sinφ, cosφ, 0])
    jnkr2 = sph_jn(nmax, k*ρ)
    # scipy.special.lpmn is for real arguments (see also clpmn)
    # + derivatives
    # m's are non-negative, cf. Bohren&Huffman (4.23)
    Pmn2 = lpmn(nmax, nmax, cosθ)
    
    # FIXME this is _wrong_ very near z axis – but it avoids nan 
    # and should give correct result exactly on z axis
    mdsinθ = m/(sinθ if sinθ else 1)
    
    Memn = np.outer(-mdsinθ[:,nx] * jnkr2[0][nx,:] * 
         Pmn2[0] * sinmφ[:,nx], θ̂ ) + np.outer(
         + sinθ * jnkr2[0][nx,:] * Pmn2[1] * cosmφ[:,nx], φ̂ )
    Momn = np.outer(mdsinθ[:,nx] * jnkr2[0][nx,:] * 
         Pmn2[0] * cosmφ[:,nx], θ̂ ) + np.outer(
         + sinθ * jnkr2[0][nx,:] * Pmn2[1] * sinmφ[:,nx], φ̂ )
    Npfac = jnkr2[0]/(k*ρ) + jnkr2[1]
    Nemn = np.outer(((n*(n+1)/(k*ρ))*jnkr2[0])[nx,:]
         * Pmn2[0] * cosmφ[:,nx], r̂) + np.outer( Npfac[nx,:] *
         (-sinθ) * Pmn2[1] * cosmφ[:,nx], θ̂  ) + np.outer(
         Npfac[nx,:] * (-mdsinθ)[:,nx] * Pmn2[0] * sinmφ[:,nx], φ̂ )
    Nomn = np.outer(((n*(n+1)/(k*ρ))*jnkr2[0])[nx,:]
         * Pmn2[0] * sinmφ[:,nx], r̂) + np.outer( Npfac[nx,:] *
         (-sinθ) * Pmn2[1] * sinmφ[:,nx], θ̂  ) + np.outer(
         Npfac[nx,:] * mdsinθ[:,nx] * Pmn2[0] * cosmφ[:,nx], φ̂ )
    return (Memn, Momn, Nemn, Nomn)



## T-matrix

In [ ]:


def M(G, ω, R, μ): # as in PRA 70, 053823, eq. (37)
    #M = eye(N)
    
    

# Examples

# Playground

In [19]:
r = np.array([[1,0,0],[2,2,2]])
R = np.array([[0,1,0],[0,1,4]])
np.linalg.norm((r-R), axis=1)
G_fs(r, R, 2e8)

array([[ 0.03302973+0.04555572j,  0.00000000+0.j        ,
         0.00000000+0.j        ,  0.00000000+0.j        ,
         0.03302973+0.04555572j,  0.00000000+0.j        ,
         0.00000000+0.j        ,  0.00000000+0.j        ,
         0.03302973+0.04555572j],
       [-0.01107202+0.02410456j, -0.00000000+0.j        ,
        -0.00000000+0.j        , -0.00000000+0.j        ,
        -0.01107202+0.02410456j, -0.00000000+0.j        ,
        -0.00000000+0.j        , -0.00000000+0.j        ,
        -0.01107202+0.02410456j]])

In [151]:
np.seterr(divide='raise')
spherical_vector_waves(5, 1, np.array([0,0,1]))


/home/necadam1/.local/lib/python3.4/site-packages/ipykernel/__main__.py:34: RuntimeWarning: invalid value encountered in multiply
/home/necadam1/.local/lib/python3.4/site-packages/ipykernel/__main__.py:37: RuntimeWarning: invalid value encountered in multiply
/home/necadam1/.local/lib/python3.4/site-packages/ipykernel/__main__.py:41: RuntimeWarning: invalid value encountered in multiply
/home/necadam1/.local/lib/python3.4/site-packages/ipykernel/__main__.py:45: RuntimeWarning: invalid value encountered in multiply


(array([[ -0.,   0.,   0.],
        [ -0.,   0.,   0.],
        [ -0.,   0.,   0.],
        [ -0.,   0.,   0.],
        [ -0.,   0.,   0.],
        [ -0.,   0.,   0.],
        [ -0.,   0.,   0.],
        [ nan,  nan,  nan],
        [ nan,  nan,  nan],
        [ nan,  nan,  nan],
        [ nan,  nan,  nan],
        [ nan,  nan,  nan],
        [  0.,  -0.,   0.],
        [ -0.,   0.,   0.],
        [ -0.,   0.,   0.],
        [ -0.,   0.,   0.],
        [ -0.,   0.,   0.],
        [ -0.,   0.,   0.],
        [  0.,   0.,  -0.],
        [  0.,   0.,  -0.],
        [  0.,   0.,  -0.],
        [  0.,   0.,  -0.],
        [  0.,   0.,  -0.],
        [  0.,   0.,  -0.],
        [  0.,   0.,   0.],
        [  0.,   0.,   0.],
        [  0.,   0.,   0.],
        [  0.,   0.,   0.],
        [  0.,   0.,   0.],
        [  0.,   0.,   0.],
        [ -0.,   0.,   0.],
        [ -0.,   0.,   0.],
        [ -0.,   0.,   0.],
        [ -0.,   0.,   0.],
        [ -0.,   0.,   0.],
        [ -0.,   0.,

In [155]:
nmax=5
k=1
r=np.array([0,0,1])

In [160]:
    # k is single scalar, r is single 3D vector
    r.shape = (3) # FIXME do proper check
    ρ = np.linalg.norm(r)
    r̂ = r/ρ
    cosθ = r̂[2]
    sinθ = np.sqrt(1-cosθ*cosθ)
    cosφ = r̂[0]/sinθ if sinθ else 0 # avoid zero division here
    sinφ = r̂[1]/sinθ if sinθ else 1
    φ = np.arctan2(sinφ,cosφ)
    # TODO can those be views so I don't have to write the indices
    # explicitly?
    m = np.arange(nmax+1)
    n = m
    sinmφ = np.sin(m*φ)
    cosmφ = np.cos(m*φ)
    θ̂ = np.array([cosθ*cosφ,cosθ*sinφ,-sinθ])
    φ̂ = np.array([-sinφ, cosφ, 0])
    jnkr2 = sph_jn(nmax, k*ρ)
    # scipy.special.lpmn is for real arguments (see also clpmn)
    # + derivatives
    # m's are non-negative, cf. Bohren&Huffman (4.23)
    Pmn2 = lpmn(nmax, nmax, cosθ)
    mdsinθ = m/(sinθ if sinθ else 1)
    Npfac = jnkr2[0]/(k*ρ) + jnkr2[1]

    Pmn2


(array([[ 1.,  1.,  1.,  1.,  1.,  1.],
        [ 0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.]]),
 array([[   0.,    1.,    3.,    6.,   10.,   15.],
        [   0.,   inf,   inf,   inf,   inf,   inf],
        [   0.,   -0.,   -6.,  -30.,  -90., -210.],
        [   0.,    0.,    0.,    0.,    0.,    0.],
        [   0.,    0.,    0.,    0.,    0.,    0.],
        [   0.,    0.,    0.,    0.,    0.,    0.]]))

In [ ]:
  # k is single scalar, r is single 3D vector
    r.shape = (3) # FIXME do proper check
    ρ = np.linalg.norm(r)
    r̂ = r/ρ
    cosθ = r̂[2]
    sinθ = np.sqrt(1-cosθ*cosθ)
    cosφ = r̂[0]/sinθ if sinθ else 0 # avoid zero division here
    sinφ = r̂[1]/sinθ if sinθ else 1
    φ = np.arctan2(sinφ,cosφ)
    # TODO can those be views so I don't have to write the indices
    # explicitly?
    m = np.arange(nmax+1)
    n = m
    sinmφ = np.sin(m*φ)
    cosmφ = np.cos(m*φ)
    θ̂ = np.array([cosθ*cosφ,cosθ*sinφ,-sinθ])
    φ̂ = np.array([-sinφ, cosφ, 0])
    jnkr2 = sph_jn(nmax, k*ρ)
    # scipy.special.lpmn is for real arguments (see also clpmn)
    # + derivatives
    # m's are non-negative, cf. Bohren&Huffman (4.23)
    Pmn2 = lpmn(nmax, nmax, cosθ)
    
    # FIXME this is _wrong_ very near z axis – but it avoids nan 
    # and should give correct result exactly on z axis
    mdsinθ = m/(sinθ if sinθ else 1)
    
    Memn = np.outer(-mdsinθ[:,nx] * jnkr2[0][nx,:] * 
         Pmn2[0] * sinmφ[:,nx], θ̂ ) + np.outer(
         + sinθ * jnkr2[0][nx,:] * Pmn2[1] * cosmφ[:,nx], φ̂ )
    Momn = np.outer(mdsinθ[:,nx] * jnkr2[0][nx,:] * 
         Pmn2[0] * cosmφ[:,nx], θ̂ ) + np.outer(
         + sinθ * jnkr2[0][nx,:] * Pmn2[1] * sinmφ[:,nx], φ̂ )
    Npfac = jnkr2[0]/(k*ρ) + jnkr2[1]
    Nemn = np.outer(((n*(n+1)/(k*ρ))*jnkr2[0])[nx,:]
         * Pmn2[0] * cosmφ[:,nx], r̂) + np.outer( Npfac[nx,:] *
         (-sinθ) * Pmn2[1] * cosmφ[:,nx], θ̂  ) + np.outer(
         Npfac[nx,:] * (-mdsinθ)[:,nx] * Pmn2[0] * sinmφ[:,nx], φ̂ )
    Nomn = np.outer(((n*(n+1)/(k*ρ))*jnkr2[0])[nx,:]
         * Pmn2[0] * sinmφ[:,nx], r̂) + np.outer( Npfac[nx,:] *
         (-sinθ) * Pmn2[1] * sinmφ[:,nx], θ̂  ) + np.outer(
         Npfac[nx,:] * mdsinθ[:,nx] * Pmn2[0] * cosmφ[:,nx], φ̂ )
    return (Memn, Momn, Nemn, Nomn)